In [1]:
from gurobipy import *

## Case 1

Choose only one supplier

In [55]:
from gurobipy import Model, GRB, quicksum

sites = ["US", "MX", "CN"]

# Exhibit A + appendices (edit if you like)
C = {"US":80.0, "MX":62.0, "CN": 59.0}                 # nominal $/lamp
Y = {"US":0.80, "MX":0.90, "CN": 0.95}                 # yield
T = {"US":0.0,  "MX":15.5, "CN": 15.0}                 # tariff $/unit
risk_pct = {"US":0.15, "MX":0.10, "CN": 0.30}          # risk premium as % of BaseLanded
damage_rate = {"US": 0.002, "MX": 0.005, "CN": 0.010}    # damage fraction of BaseLanded
delay_cost_per_day = {"US": 0.055, "MX": 0.053, "CN": 0.631}  # $/lamp/day delayed
expected_delay_days = {"US": 1.0, "MX": 5.0, "CN": 15.0}      # expected delayed days

K = {"US": 400000, "MX": 400000, "CN": 400000}        # capacity (good units)
D = 300000                                            # demand (good units)
F = {"US":0.0, "MX": 0.0, "CN":0.0}                    # activation $

# Effective unit costs v_i
BaseLanded, Risk, Damage, Delay, v = {}, {}, {}, {}, {}
for i in sites:
    BaseLanded[i] = (C[i] + T[i]) / Y[i]
    Risk[i] = risk_pct[i] * BaseLanded[i]
    Damage[i] = damage_rate[i] * BaseLanded[i]
    Delay[i] = delay_cost_per_day[i] * expected_delay_days[i]
    v[i] = BaseLanded[i] + Risk[i] + Damage[i] + Delay[i]

m = Model("Sourcing_FreeShares")
x = {i: m.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name=f"x_{i}") for i in sites}
y = {i: m.addVar(vtype=GRB.BINARY, name=f"y_{i}") for i in sites}
m.setObjective(quicksum(v[i]*x[i] for i in sites) + quicksum(F[i]*y[i] for i in sites), GRB.MINIMIZE)

# Demand
m.addConstr(quicksum(x[i] * Y[i] * (1 - damage_rate[i]) for i in sites) == D, name="Demand")
# Capacity + linking
for i in sites:
    m.addConstr(x[i] <= K[i]*y[i], name=f"Cap_{i}")
# Policy
m.addConstr(quicksum(y[i] for i in sites) == 1, name="SupplierCount")

m.Params.OutputFlag = 0 
m.optimize()

if m.status == GRB.OPTIMAL:
    print("\n=== Optimal plan (free shares) ===")
    total = m.objVal
    for i in sites:
        xi = x[i].X
        print(f"{i}: x={xi:,.0f}  share={100*xi/D:5.1f}%  active={int(y[i].X)}  unit=${v[i]:.3f}")
    print(f"Total cost = ${total:,.2f}\n")

    print("-- Per-unit cost components --")
    for i in sites:
        print(f"{i}: Base={BaseLanded[i]:.3f}  Risk={Risk[i]:.3f}  Damage={Damage[i]:.3f}  Delay={Delay[i]:.3f}  => v={v[i]:.3f}")
else:
    print(f"Status: {m.status}")



=== Optimal plan (free shares) ===
US: x=0  share=  0.0%  active=0  unit=$115.255
MX: x=335,008  share=111.7%  active=1  unit=$95.418
CN: x=0  share=  0.0%  active=0  unit=$111.507
Total cost = $31,965,754.70

-- Per-unit cost components --
US: Base=100.000  Risk=15.000  Damage=0.200  Delay=0.055  => v=115.255
MX: Base=86.111  Risk=8.611  Damage=0.431  Delay=0.265  => v=95.418
CN: Base=77.895  Risk=23.368  Damage=0.779  Delay=9.465  => v=111.507


## Case 2

Dynamic allocation of suppliers

In [56]:
from gurobipy import Model, GRB, quicksum

sites = ["US", "MX", "CN"]

# Exhibit A + risk/delay/damage assumptions
C = {"US":80.0, "MX":62.0, "CN":59.0}
Y = {"US":0.80, "MX":0.90, "CN":0.95}
T = {"US":0.0, "MX":15.5, "CN":15.0}
risk_pct = {"US": 0.15, "MX": 0.10, "CN":0.30}
damage_rate = {"US": 0.002, "MX":0.005, "CN":0.010}
delay_cost_per_day = {"US": 0.055, "MX": 0.053, "CN": 0.631}
expected_delay_days = {"US": 1.0, "MX": 2.0, "CN":15.0}
L = {"US": 7, "MX": 10, "CN": 40}

# Scaled capacities (for D=300k)
K = {"US": 200000, "MX": 200000, "CN": 200000}
D = 300000
F = {"US":0.0, "MX": 0.0, "CN": 0.0}
R1 = 0.33 * D   # >= 33% must arrive within 7 days (US only)
R2 = 0.5 * D   # >= 50% must arrive within 10 days (US+MX)
# R3 = D is already enforced by the demand constraint

# Build per-unit costs
BaseLanded, Risk, Damage, Delay, v = {}, {}, {}, {}, {}
for i in sites:
    BaseLanded[i] = (C[i] + T[i]) / Y[i]
    Risk[i] = risk_pct[i] * BaseLanded[i]
    Damage[i] = damage_rate[i] * BaseLanded[i]
    Delay[i] = delay_cost_per_day[i] * expected_delay_days[i]
    v[i] = BaseLanded[i] + Risk[i] + Damage[i] + Delay[i]

# Model
m = Model("Tesla_Sourcing")
x = {i: m.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name=f"x_{i}") for i in sites}
y = {i: m.addVar(vtype=GRB.BINARY, name=f"y_{i}") for i in sites}

m.setObjective(quicksum(v[i] * x[i] for i in sites) + quicksum(F[i] * y[i] for i in sites), GRB.MINIMIZE)

# Constraints
m.addConstr(quicksum(x[i] for i in sites) == D, name="Demand")
# Add multi-level lead-time constraints
m.addConstr(quicksum(x[i] for i in sites if L[i] <= 7) >= R1, name="Lead_7day")
m.addConstr(quicksum(x[i] for i in sites if L[i] <= 10) >= R2, name="Lead_10day")
for i in sites:
    m.addConstr(x[i] <= K[i]*y[i], name=f"Cap_{i}")
m.addConstr(quicksum(y[i] for i in sites) <= 3, name="SupplierCount")
m.Params.OutputFlag = 0
m.optimize()
if m.status == GRB.OPTIMAL:
    print("\n=== Optimal sourcing ===")
    total = m.objVal
    for i in sites:
        xi = x[i].X
        print(f"{i}: {xi:,.0f} units ({100*xi/D:4.1f}%), active={int(y[i].X)}, unit=${v[i]:.3f}")
    print(f"Total cost = ${total:,.2f}")



=== Optimal sourcing ===
US: 99,000 units (33.0%), active=1, unit=$115.255
MX: 200,000 units (66.7%), active=1, unit=$95.259
CN: 1,000 units ( 0.3%), active=1, unit=$111.507
Total cost = $30,573,507.66


## Case 3

Considering the lead time, the damage rate, production yield.

In [57]:
from gurobipy import Model, GRB, quicksum

sites = ["US", "MX", "CN"]

# Exhibit A + risk/delay/damage assumptions
C = {"US":80.0, "MX":62.0, "CN":59.0}
Y = {"US":0.80, "MX":0.90, "CN":0.95}
T = {"US":0.0, "MX":15.5, "CN":15.0}
risk_pct = {"US": 0.15, "MX": 0.10, "CN":0.30}
damage_rate = {"US": 0.002, "MX":0.005, "CN":0.010}
delay_cost_per_day = {"US": 0.055, "MX": 0.053, "CN": 0.631}
expected_delay_days = {"US": 1.0, "MX": 2.0, "CN":15.0}
L = {"US": 7, "MX": 10, "CN": 40}

# Scaled capacities (for D=300k)
K = {"US": 200000, "MX": 200000, "CN": 200000}
D = 300000
F = {"US":0.0, "MX": 0.0, "CN": 0.0}
R1 = 0.33 * D   # >= 33% must arrive within 7 days (US only)
R2 = 0.5 * D   # >= 50% must arrive within 10 days (US+MX)
# R3 = D is already enforced by the demand constraint

# Build per-unit costs
BaseLanded, Risk, Damage, Delay, v = {}, {}, {}, {}, {}
for i in sites:
    BaseLanded[i] = (C[i] + T[i]) / Y[i]
    Risk[i] = risk_pct[i] * BaseLanded[i]
    Damage[i] = damage_rate[i] * BaseLanded[i]
    Delay[i] = delay_cost_per_day[i] * expected_delay_days[i]
    v[i] = BaseLanded[i] + Risk[i] + Damage[i] + Delay[i]

# Model
m = Model("Tesla_Sourcing")
x = {i: m.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name=f"x_{i}") for i in sites}
y = {i: m.addVar(vtype=GRB.BINARY, name=f"y_{i}") for i in sites}

m.setObjective(quicksum(v[i] * x[i] for i in sites) + quicksum(F[i] * y[i] for i in sites), GRB.MINIMIZE)

# Constraints
m.addConstr(quicksum(x[i] * Y[i] * (1 - damage_rate[i]) for i in sites) == D, name="Demand")
# Add multi-level lead-time constraints
m.addConstr(quicksum(x[i] for i in sites if L[i] <= 7) >= R1, name="Lead_7day")
m.addConstr(quicksum(x[i] for i in sites if L[i] <= 10) >= R2, name="Lead_10day")
for i in sites:
    m.addConstr(x[i] <= K[i]*y[i], name=f"Cap_{i}")
m.addConstr(quicksum(y[i] for i in sites) <= 3, name="SupplierCount")
m.Params.OutputFlag = 0
m.optimize()
m.Params.OutputFlag = 0
if m.status == GRB.OPTIMAL:
    print("\n=== Optimal sourcing ===")
    total = m.objVal
    for i in sites:
        xi = x[i].X
        print(f"{i}: {xi:,.0f} units ({100*xi/D:4.1f}%), active={int(y[i].X)}, unit=${v[i]:.3f}")
    print(f"Total cost = ${total:,.2f}")



=== Optimal sourcing ===
US: 99,000 units (33.0%), active=1, unit=$115.255
MX: 200,000 units (66.7%), active=1, unit=$95.259
CN: 44,507 units (14.8%), active=1, unit=$111.507
Total cost = $35,424,795.89


# Case 4

Updated cost function

In [73]:
from gurobipy import Model, GRB, quicksum

sites = ["US", "MX", "CN"]

# ==== Exhibit A cost breakdown ($/lamp) ====
raw  = {"US":40.0, "MX":35.0, "CN":30.0}
lab  = {"US":12.0, "MX": 8.0, "CN": 4.0}
ind  = {"US":10.0, "MX": 8.0, "CN": 4.0}
pack = {"US": 9.0, "MX": 7.0, "CN":12.0}
elec = {"US": 4.0, "MX": 3.0, "CN": 4.0}
depr = {"US": 5.0, "MX": 1.0, "CN": 5.0}
tariff = {"US":0.0, "MX":15.5, "CN":15.0}

# Quality & logistics
Y = {"US":0.80, "MX":0.90, "CN":0.95}                 # production yield
damage = {"US":0.002, "MX":0.005, "CN":0.010}         # in-transit damage on good units

# Risk (as fraction of good-unit cost), Delay ($/lamp/day * expected days)
risk_pct = {"US":0.15, "MX":0.10, "CN":0.30}          # can tune
delay_cost_per_day = {"US":0.055, "MX":0.053, "CN":0.631}
expected_delay_days = {"US":1.0, "MX":2.0, "CN":15.0}

# Lead times (days) and service windows
L = {"US":7, "MX":10, "CN":40}
D = 300000
R1 = 0.33 * D     # >= 33% must be from L<=7
R2 = 0.50 * D     # >= 50% must be from L<=10

# Capacities (good units started; we model on x (started), so these are "starts" caps)
K = {"US":200000, "MX":200000, "CN":200000}
F = {"US":0.0, "MX":0.0, "CN":0.0}                    # activation cost

# ---- Build per-site costs ----
C_good = {i: raw[i] + lab[i] + ind[i] + pack[i] + elec[i] + depr[i] + tariff[i] for i in sites}
C_bad  = {i: raw[i] + lab[i] + elec[i] for i in sites}

# Risk & delay per delivered unit
Risk = {i: risk_pct[i] * C_good[i] for i in sites}
Delay = {i: delay_cost_per_day[i] * expected_delay_days[i] for i in sites}

# ---- Model ----
m = Model("Tesla_Sourcing_Final")

# Decision variables
x = {i: m.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name=f"x_{i}") for i in sites}  # units started
y = {i: m.addVar(vtype=GRB.BINARY, name=f"y_{i}") for i in sites}              # activate

# Helper (delivered good units)
delivered = {i: x[i] * Y[i] * (1.0 - damage[i]) for i in sites}

# Objective
good_cost  = quicksum(C_good[i] * (x[i] * Y[i])          for i in sites)
bad_cost   = quicksum(C_bad[i]  * (x[i] * (1.0 - Y[i]))  for i in sites)
risk_cost  = quicksum(Risk[i]   * delivered[i]           for i in sites)
delay_cost = quicksum(Delay[i]  * delivered[i]           for i in sites)
act_cost   = quicksum(F[i] * y[i]                        for i in sites)

m.setObjective(good_cost + bad_cost + risk_cost + delay_cost + act_cost, GRB.MINIMIZE)

# Constraints
m.addConstr(quicksum(delivered[i] for i in sites) == D, name="Demand")

m.addConstr(quicksum(x[i] for i in sites if L[i] <= 7)  >= R1, name="Lead_7day")
m.addConstr(quicksum(x[i] for i in sites if L[i] <= 10) >= R2, name="Lead_10day")

for i in sites:
    m.addConstr(x[i] <= K[i] * y[i], name=f"Cap_{i}")

m.addConstr(quicksum(y[i] for i in sites) <= 3, name="SupplierCount")

m.Params.OutputFlag = 0
m.optimize()

if m.status == GRB.OPTIMAL:
    print("\n=== Optimal sourcing (good/bad split) ===")
    total = m.objVal
    delivered_total = sum(delivered[i].getValue() for i in sites)
    for i in sites:
        xi = x[i].X
        di = delivered[i].getValue()
        print(f"{i}: starts={xi:,.0f}, delivered={di:,.0f} "
              f"({100*di/D:4.1f}%), active={int(y[i].X)}")
    print(f"Delivered total = {delivered_total:,.0f} (target {D:,.0f})")
    print(f"Total cost = ${total:,.2f}")



=== Optimal sourcing (good/bad split) ===
US: starts=151,428, delivered=120,900 (40.3%), active=1
MX: starts=200,000, delivered=179,100 (59.7%), active=1
CN: starts=0, delivered=0 ( 0.0%), active=1
Delivered total = 300,000 (target 300,000)
Total cost = $29,121,833.85


# Final Result

Considering all the potential factors, the MILP problem can be solved as

In [83]:
from gurobipy import Model, GRB, quicksum

sites = ["US", "MX", "CN"]

# ==== Exhibit A cost breakdown ($/lamp) ====
raw  = {"US":40.0, "MX":35.0, "CN":30.0}
lab  = {"US":12.0, "MX": 8.0, "CN": 4.0}
ind  = {"US":10.0, "MX": 8.0, "CN": 4.0}
pack = {"US": 9.0, "MX": 7.0, "CN":12.0}
elec = {"US": 4.0, "MX": 3.0, "CN": 4.0}
depr = {"US": 5.0, "MX": 1.0, "CN": 5.0}
tariff = {"US":0.0, "MX":15.5, "CN":15.0}
ESG = {"US":0.15, "MX":0.20, "CN":0.25}

# Quality & logistics
Y = {"US":0.80, "MX":0.90, "CN": 0.95}                 # production yield
damage = {"US":0.002, "MX":0.005, "CN": 0.010}         # in-transit damage on good units

# Risk (as fraction of good-unit cost), Delay ($/lamp/day * expected days)
risk_pct = {"US": 0.15, "MX": 0.10, "CN": 0.30}          # can tune
delay_cost_per_day = {"US": 0.055, "MX": 0.053, "CN": 0.631}
expected_delay_days = {"US": 1.0, "MX": 2.0, "CN": 15.0}

# Lead times (days) and service windows
L = {"US": 7, "MX": 10, "CN": 40}
D = 300000
R1 = 0.333 * D     # >= 33% must be from L<=7
R2 = 0.5 * D     # >= 50% must be from L<=10

# Capacities (good units started; we model on x (started), so these are "starts" caps)
K = {"US": 100000, "MX": 150000, "CN": 300000}
# F = {"US": 0.0, "MX": 0.0, "CN":0.0}                    # activation cost

# ---- Build per-site costs ----
C_good = {i: raw[i] + lab[i] + ind[i] + pack[i] + elec[i] + depr[i] + tariff[i] for i in sites}
C_bad  = {i: raw[i] + lab[i] + elec[i] + depr[i] for i in sites}

# Risk & delay per delivered unit
Risk = {i: risk_pct[i] * C_good[i] for i in sites}
Delay = {i: delay_cost_per_day[i] * expected_delay_days[i] for i in sites}

# ---- Model ----
m = Model("Tesla_Sourcing_Final")

# Decision variables
x = {i: m.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name=f"x_{i}") for i in sites}  # units started
y = {i: m.addVar(vtype=GRB.BINARY, name=f"y_{i}") for i in sites}              # activate

# Helper (delivered good units)
delivered = {i: x[i] * Y[i] * (1.0 - damage[i]) for i in sites}

# Objective
good_cost  = quicksum(C_good[i] * (x[i] * Y[i])          for i in sites)
bad_cost   = quicksum(C_bad[i]  * (x[i] * (1.0 - Y[i]))  for i in sites)
risk_cost  = quicksum(Risk[i]   * delivered[i]           for i in sites)
delay_cost = quicksum(Delay[i]  * delivered[i]           for i in sites)
# act_cost   = quicksum(F[i] * y[i]                        for i in sites)

esg_cost = quicksum(ESG[i] * delivered[i] for i in sites)
m.setObjective(good_cost + bad_cost + risk_cost + delay_cost + esg_cost , GRB.MINIMIZE)

# Constraints
m.addConstr(quicksum(delivered[i] for i in sites) == D, name="Demand")

m.addConstr(quicksum(x[i] for i in sites if L[i] <= 7) >= R1, name="Lead_7day")
m.addConstr(quicksum(x[i] for i in sites if L[i] <= 10) >= R2, name="Lead_10day")

for i in sites:
    m.addConstr(x[i] <= K[i] * y[i], name=f"Cap_{i}")

m.addConstr(quicksum(y[i] for i in sites) <= 3, name="SupplierCount")

m.Params.OutputFlag = 0
m.optimize()

if m.status == GRB.OPTIMAL:
    print("\n=== Optimal sourcing (good/bad split) ===")
    total = m.objVal
    delivered_total = sum(delivered[i].getValue() for i in sites)
    for i in sites:
        xi = x[i].X
        di = delivered[i].getValue()
        print(f"{i}: starts={xi:,.0f}, delivered={di:,.0f} "
              f"({100*di/D:4.1f}%), active={int(y[i].X)}")
    print(f"Delivered total = {delivered_total:,.0f} (target {D:,.0f})")
    print(f"Total cost = ${total:,.2f}")


=== Optimal sourcing (good/bad split) ===
US: starts=100,000, delivered=79,840 (26.6%), active=1
MX: starts=150,000, delivered=134,325 (44.8%), active=1
CN: starts=91,265, delivered=85,835 (28.6%), active=1
Delivered total = 300,000 (target 300,000)
Total cost = $30,195,663.28
